# Purpose
The purpose of this notebook is to create a postgresql database using the 4 csv files from the data folder.

## Requirements
You may either create a local.py file in the src folder, or simply input your local sql database information in 'Credentials' cell below (after the module import cell)

Your local postgresql must have a database called 'match_finder'

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from sqlalchemy import create_engine
from src import local
from src import functions

#### PostgreSQL connection set up

In [2]:
# Credentials
USER = local.user 
PASS = local.password
HOST = local.host
PORT = local.port

#create engine
engine = create_engine(f'postgresql://{USER}:{PASS}@{HOST}:{PORT}/match_finder')

## 5 Primary tables
- events
- bouts
- fighters
- rounds tables:
    - general
    - strikes

### Events

In [3]:
events = pd.read_csv('../../data/ufcstats_data/events.csv')
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525 entries, 0 to 524
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Date        525 non-null    object
 1   Location    525 non-null    object
 2   Attendance  504 non-null    object
 3   name        525 non-null    object
 4   link        525 non-null    object
dtypes: object(5)
memory usage: 20.6+ KB


#### Lower case all columns

In [4]:
events.columns = events.columns.map(lambda x: x.lower())

#### Get id

In [5]:
events['id'] = events['link'].map(functions.get_id)

#drop link column
events.drop('link', axis=1, inplace=True)
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525 entries, 0 to 524
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        525 non-null    object
 1   location    525 non-null    object
 2   attendance  504 non-null    object
 3   name        525 non-null    object
 4   id          525 non-null    object
dtypes: object(5)
memory usage: 20.6+ KB


##### Drop duplicates

In [6]:
events.drop_duplicates(inplace=True)
events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 525 entries, 0 to 524
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        525 non-null    object
 1   location    525 non-null    object
 2   attendance  504 non-null    object
 3   name        525 non-null    object
 4   id          525 non-null    object
dtypes: object(5)
memory usage: 24.6+ KB


#### to sql

In [7]:
events.to_sql('events', engine, index=False, if_exists='replace')

### Bouts

In [8]:
bouts = pd.read_csv('../../data/ufcstats_data/bouts.csv')
bouts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5688 entries, 0 to 5687
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Method      5688 non-null   object
 1   Round       5688 non-null   int64 
 2   Time        5688 non-null   object
 3   Timeformat  5688 non-null   object
 4   Referee     5660 non-null   object
 5   details     2535 non-null   object
 6   event_link  5688 non-null   object
 7   link        5688 non-null   object
dtypes: int64(1), object(7)
memory usage: 355.6+ KB


#### Lower case all columns

In [9]:
bouts.columns = bouts.columns.map(lambda x: x.lower())

In [10]:
bouts.columns = ['method', 
                 'final_round', 
                 'time', 
                 'timeformat', 
                 'referee', 
                 'details', 
                 'event_link',
                 'link']

#### Get id

In [11]:
bouts['id'] = bouts['link'].map(functions.get_id)
bouts['event_id'] = bouts['event_link'].map(functions.get_id)
#drop link column
bouts.drop(['link', 'event_link'], axis=1, inplace=True)
bouts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5688 entries, 0 to 5687
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   method       5688 non-null   object
 1   final_round  5688 non-null   int64 
 2   time         5688 non-null   object
 3   timeformat   5688 non-null   object
 4   referee      5660 non-null   object
 5   details      2535 non-null   object
 6   id           5688 non-null   object
 7   event_id     5688 non-null   object
dtypes: int64(1), object(7)
memory usage: 355.6+ KB


##### Drop duplicates

In [13]:
bouts.drop_duplicates(inplace=True)
bouts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5681 entries, 0 to 5687
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   method       5681 non-null   object
 1   final_round  5681 non-null   int64 
 2   time         5681 non-null   object
 3   timeformat   5681 non-null   object
 4   referee      5653 non-null   object
 5   details      2530 non-null   object
 6   id           5681 non-null   object
 7   event_id     5681 non-null   object
dtypes: int64(1), object(7)
memory usage: 399.4+ KB


#### Send to sql

In [14]:
bouts.to_sql('bouts', engine, index=False, if_exists='replace')

### Fighters

In [15]:
fighters = pd.read_csv('../../data/ufcstats_data/fighters.csv')
fighters

,Height,Weight,Reach,STANCE,DOB,name,link
0,"6' 3""",257 lbs.,--,Orthodox,"Jun 30, 1972",Fabiano Scherner,http://www.ufcstats.com/fighter-details/0bc7e6...
1,"6' 2""",238 lbs.,--,Orthodox,"Mar 01, 1979",Alexandre Dantas,http://www.ufcstats.com/fighter-details/8dc46e...
2,"5' 10""",170 lbs.,"74""",Orthodox,"Mar 03, 1980",Kevin Burns,http://www.ufcstats.com/fighter-details/a6d8bf...
3,"5' 7""",135 lbs.,--,Southpaw,"May 11, 1986",Joseph Sandoval,http://www.ufcstats.com/fighter-details/696002...
4,"5' 7""",135 lbs.,"70""",Orthodox,"Sep 09, 1986",Jose Aldo,http://www.ufcstats.com/fighter-details/d0f395...
...,...,...,...,...,...,...,...
2040,"6' 3""",227 lbs.,--,Orthodox,"Sep 16, 1968",Roberto Traven,http://www.ufcstats.com/fighter-details/20ec00...
2041,"5' 9""",155 lbs.,"70""",Orthodox,"Sep 12, 1984",Paul Kelly,http://www.ufcstats.com/fighter-details/aac5ac...
2042,"6' 1""",205 lbs.,"75""",Orthodox,"Jun 22, 1963",Randy Couture,http://www.ufcstats.com/fighter-details/0aa925...
2043,"6' 1""",240 lbs.,"77""",Orthodox,"Jul 28, 1982",Cain Velasquez,http://www.ufcstats.com/fighter-details/0ff11c...


#### Lower case all columns

In [16]:
fighters.columns = fighters.columns.map(lambda x: x.lower())

#### Get id

In [17]:
fighters['id'] = fighters['link'].map(functions.get_id)
#drop link column
fighters.drop(['link'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2045 entries, 0 to 2044
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   height  2045 non-null   object
 1   weight  2045 non-null   object
 2   reach   2045 non-null   object
 3   stance  1972 non-null   object
 4   dob     2045 non-null   object
 5   name    2045 non-null   object
 6   id      2045 non-null   object
dtypes: object(7)
memory usage: 112.0+ KB


##### Drop duplicates

In [21]:
fighters.drop_duplicates(inplace=True)
fighters.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2045 entries, 0 to 2044
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   height  2045 non-null   object
 1   weight  2045 non-null   object
 2   reach   2045 non-null   object
 3   stance  1972 non-null   object
 4   dob     2045 non-null   object
 5   name    2045 non-null   object
 6   id      2045 non-null   object
dtypes: object(7)
memory usage: 127.8+ KB


#### Send to sql

In [20]:
fighters.to_sql('fighters', engine, index=False, if_exists='replace')

### Rounds tables
#### Combine General and Strikes

##### Import and get bout and fighter IDs

In [16]:
general = pd.read_csv('../../data/ufcstats_data/general.csv')
strikes= pd.read_csv('../../data/ufcstats_data/strikes.csv')

# rename bout_id column to bout_link
general.rename(mapper={'bout_id': 'bout_link'}, axis=1, inplace=True)
strikes.rename(mapper={'bout_id': 'bout_link'}, axis=1, inplace=True)

general = functions.get_all_ids(general, event=False)
strikes = functions.get_all_ids(strikes, event=False)

general.head()

,fighter,kd,sig_str,sig_str_prcnt,total_str,td_count,td_prcnt,sub_att,pass,rev,round,bout_link,outcome,fighter_link,fighter_id,bout_id
0,Robert Whittaker,0,11 of 30,36%,13 of 32,0 of 2,0%,0,0,0,1,http://www.ufcstats.com/fight-details/11f715fa...,W,http://www.ufcstats.com/fighter-details/e1147d...,e1147d3d2dabe1ce,11f715fa5e825e51
1,Robert Whittaker,1,15 of 30,50%,42 of 57,0 of 0,0%,0,1,0,2,http://www.ufcstats.com/fight-details/11f715fa...,W,http://www.ufcstats.com/fighter-details/e1147d...,e1147d3d2dabe1ce,11f715fa5e825e51
2,Robert Whittaker,0,13 of 32,40%,13 of 32,0 of 2,0%,0,0,0,3,http://www.ufcstats.com/fight-details/11f715fa...,W,http://www.ufcstats.com/fighter-details/e1147d...,e1147d3d2dabe1ce,11f715fa5e825e51
3,Robert Whittaker,0,13 of 34,38%,14 of 35,0 of 2,0%,0,0,0,4,http://www.ufcstats.com/fight-details/11f715fa...,W,http://www.ufcstats.com/fighter-details/e1147d...,e1147d3d2dabe1ce,11f715fa5e825e51
4,Robert Whittaker,0,17 of 31,54%,18 of 32,2 of 7,28%,0,0,0,5,http://www.ufcstats.com/fight-details/11f715fa...,W,http://www.ufcstats.com/fighter-details/e1147d...,e1147d3d2dabe1ce,11f715fa5e825e51


In [17]:
strikes

,fighter,sig_str,sig_str_prcnt,head,body,leg,distance,clinch,ground,round,bout_link,outcome,fighter_link,fighter_id,bout_id
0,Robert Whittaker,11 of 30,36%,4 of 22,1 of 2,6 of 6,10 of 29,1 of 1,0 of 0,1,http://www.ufcstats.com/fight-details/11f715fa...,W,http://www.ufcstats.com/fighter-details/e1147d...,e1147d3d2dabe1ce,11f715fa5e825e51
1,Robert Whittaker,15 of 30,50%,7 of 22,1 of 1,7 of 7,10 of 23,0 of 0,5 of 7,2,http://www.ufcstats.com/fight-details/11f715fa...,W,http://www.ufcstats.com/fighter-details/e1147d...,e1147d3d2dabe1ce,11f715fa5e825e51
2,Robert Whittaker,13 of 32,40%,8 of 26,0 of 1,5 of 5,11 of 30,2 of 2,0 of 0,3,http://www.ufcstats.com/fight-details/11f715fa...,W,http://www.ufcstats.com/fighter-details/e1147d...,e1147d3d2dabe1ce,11f715fa5e825e51
3,Robert Whittaker,13 of 34,38%,7 of 26,1 of 2,5 of 6,12 of 31,1 of 3,0 of 0,4,http://www.ufcstats.com/fight-details/11f715fa...,W,http://www.ufcstats.com/fighter-details/e1147d...,e1147d3d2dabe1ce,11f715fa5e825e51
4,Robert Whittaker,17 of 31,54%,6 of 20,4 of 4,7 of 7,14 of 26,3 of 5,0 of 0,5,http://www.ufcstats.com/fight-details/11f715fa...,W,http://www.ufcstats.com/fighter-details/e1147d...,e1147d3d2dabe1ce,11f715fa5e825e51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26239,Art Jimmerson,0 of 0,0%,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,1,http://www.ufcstats.com/fight-details/cecdc0da...,L,http://www.ufcstats.com/fighter-details/a5c53b...,a5c53b3ddb31cc7d,cecdc0da584274b9
26240,Kevin Rosier,15 of 27,55%,12 of 23,3 of 4,0 of 0,4 of 10,4 of 9,7 of 8,1,http://www.ufcstats.com/fight-details/2d2bbc86...,W,http://www.ufcstats.com/fighter-details/598a58...,598a58db87b890ee,2d2bbc86e941e05c
26241,Zane Frazier,12 of 28,42%,7 of 19,3 of 6,2 of 3,0 of 7,10 of 19,2 of 2,1,http://www.ufcstats.com/fight-details/2d2bbc86...,L,http://www.ufcstats.com/fighter-details/d3711d...,d3711d3784b76255,2d2bbc86e941e05c
26242,Gerard Gordeau,3 of 5,60%,3 of 5,0 of 0,0 of 0,1 of 3,0 of 0,2 of 2,1,http://www.ufcstats.com/fight-details/567a09fd...,W,http://www.ufcstats.com/fighter-details/279093...,279093302a6f44b3,567a09fd200cfa05


##### Convert round to string

In [18]:
strikes['round'] = strikes['round'].map(str)
general['round'] = general['round'].map(str)

##### Create unique ID for each row

In [19]:
strikes['round_id'] = strikes['bout_id'] + strikes['fighter_id'] + strikes['round']
general['round_id'] = general['bout_id'] + general['fighter_id'] + general['round']

##### Merge

In [20]:
rounds = pd.merge(strikes, general, copy=False)
rounds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26364 entries, 0 to 26363
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   fighter        26364 non-null  object
 1   sig_str        26364 non-null  object
 2   sig_str_prcnt  26364 non-null  object
 3   head           26364 non-null  object
 4   body           26364 non-null  object
 5   leg            26364 non-null  object
 6   distance       26364 non-null  object
 7   clinch         26364 non-null  object
 8   ground         26364 non-null  object
 9   round          26364 non-null  object
 10  bout_link      26364 non-null  object
 11  outcome        26364 non-null  object
 12  fighter_link   26364 non-null  object
 13  fighter_id     26364 non-null  object
 14  bout_id        26364 non-null  object
 15  round_id       26364 non-null  object
 16  kd             26364 non-null  int64 
 17  total_str      26364 non-null  object
 18  td_count       26364 non-n

##### Reorganize columns

In [21]:
rounds.columns

Index(['fighter', 'sig_str', 'sig_str_prcnt', 'head', 'body', 'leg',
       'distance', 'clinch', 'ground', 'round', 'bout_link', 'outcome',
       'fighter_link', 'fighter_id', 'bout_id', 'round_id', 'kd', 'total_str',
       'td_count', 'td_prcnt', 'sub_att', 'pass', 'rev'],
      dtype='object')

In [22]:
rounds = rounds.loc[:, ['fighter', 'sig_str', 'head', 'body', 'leg', 'distance', 
                        'clinch', 'ground', 'kd', 'total_str','td_count', 'sub_att', 
                        'pass', 'rev', 'outcome', 'bout_id', 'fighter_id', 'round']]

rounds.columns

Index(['fighter', 'sig_str', 'head', 'body', 'leg', 'distance', 'clinch',
       'ground', 'kd', 'total_str', 'td_count', 'sub_att', 'pass', 'rev',
       'outcome', 'bout_id', 'fighter_id', 'round'],
      dtype='object')

##### Abreviate column names

In [23]:
rounds.columns = ['fighter', 'ss', 'h_ss', 'b_ss', 'l_ss', 'd_ss', 
                  'c_ss', 'g_ss', 'kd', 'ts', 'td', 'sba', 'ps', 'rev', 
                  'outcome', 'bout_id', 'fighter_id', 'round']

In [24]:
rounds.drop_duplicates(inplace=True, ignore_index=True)

##### parse values into attempts (_a suffix) and successes (_s suffix)

In [25]:
to_be_formatted = ['ss', 'h_ss', 'b_ss', 'l_ss', 'd_ss',
                   'c_ss', 'g_ss', 'ts', 'td']

for column in to_be_formatted:
    # Clear whitespace
    rounds[column] = rounds[column].map(lambda x: x.strip())
    rounds[column+'_s'] = rounds[column].map(functions.get_successful)
    rounds[column+'_a'] = rounds[column].map(functions.get_attempts)

In [26]:
rounds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26214 entries, 0 to 26213
Data columns (total 36 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   fighter     26214 non-null  object
 1   ss          26214 non-null  object
 2   h_ss        26214 non-null  object
 3   b_ss        26214 non-null  object
 4   l_ss        26214 non-null  object
 5   d_ss        26214 non-null  object
 6   c_ss        26214 non-null  object
 7   g_ss        26214 non-null  object
 8   kd          26214 non-null  int64 
 9   ts          26214 non-null  object
 10  td          26214 non-null  object
 11  sba         26214 non-null  int64 
 12  ps          26214 non-null  int64 
 13  rev         26214 non-null  int64 
 14  outcome     26214 non-null  object
 15  bout_id     26214 non-null  object
 16  fighter_id  26214 non-null  object
 17  round       26214 non-null  object
 18  ss_s        26214 non-null  int64 
 19  ss_a        26214 non-null  int64 
 20  h_ss_s

##### Reformat columns to remove old rows

In [27]:
rounds = rounds.loc[:,['fighter', 'kd', 'sba', 'ps', 'rev', 
              'ss_s', 'ss_a', 'h_ss_s', 'h_ss_a', 
              'b_ss_s', 'b_ss_a', 'l_ss_s', 'l_ss_a', 
              'd_ss_s', 'd_ss_a', 'c_ss_s', 'c_ss_a', 
              'g_ss_s', 'g_ss_a', 'ts_s', 'ts_a', 
              'td_s', 'td_a', 'outcome', 'bout_id', 
              'fighter_id', 'round', ]]

#### Send to sql

In [28]:
rounds.to_sql('rounds', engine, index=False, if_exists='replace')

### Advanced Statistics tables

Read from csv

In [29]:
body_ss_stats = pd.read_csv('../../data/ufcstats_data/advanced_stats/body_ss_stats.csv')
head_ss_stats = pd.read_csv('../../data/ufcstats_data/advanced_stats/head_ss_stats.csv')
leg_ss_stats = pd.read_csv('../../data/ufcstats_data/advanced_stats/leg_ss_stats.csv')
distance_ss_stats = pd.read_csv('../../data/ufcstats_data/advanced_stats/distance_ss_stats.csv')
clinch_ss_stats = pd.read_csv('../../data/ufcstats_data/advanced_stats/clinch_ss_stats.csv')
ground_ss_stats = pd.read_csv('../../data/ufcstats_data/advanced_stats/ground_ss_stats.csv')
sig_str_stats = pd.read_csv('../../data/ufcstats_data/advanced_stats/sig_str_stats.csv')
grappling_stats = pd.read_csv('../../data/ufcstats_data/advanced_stats/grappling_stats.csv')
takedown_stats = pd.read_csv('../../data/ufcstats_data/advanced_stats/takedown_stats.csv')
total_strikes_stats = pd.read_csv('../../data/ufcstats_data/advanced_stats/total_strike_stats.csv')
knockdown_stats = pd.read_csv('../../data/ufcstats_data/advanced_stats/knockdown_stats.csv')

Send to SQL

In [30]:
body_ss_stats.to_sql('body_ss_stats', engine, index=False, if_exists='replace')
head_ss_stats.to_sql('head_ss_stats', engine, index=False, if_exists='replace')
leg_ss_stats.to_sql('leg_ss_stats', engine, index=False, if_exists='replace')
distance_ss_stats.to_sql('distance_ss_stats', engine, index=False, if_exists='replace')
clinch_ss_stats.to_sql('clinch_ss_stats', engine, index=False, if_exists='replace')
ground_ss_stats.to_sql('ground_ss_stats', engine, index=False, if_exists='replace')
sig_str_stats.to_sql('sig_str_stats', engine, index=False, if_exists='replace')
grappling_stats.to_sql('grappling_stats', engine, index=False, if_exists='replace')
takedown_stats.to_sql('takedown_stats', engine, index=False, if_exists='replace')
total_strikes_stats.to_sql('total_strike_stats', engine, index=False, if_exists='replace')
knockdown_stats.to_sql('knockdown_stats', engine, index=False, if_exists='replace')